#GEOCODE WITH GEOPY

1- Authorizing colab access to get files inside the drive.

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

2 - Installing necessary libraries for the code.

In [ ]:
!pip install unidecode
!pip install geopy
!pip install folium

3 - Importing necessary libraries for the code.

In [ ]:
import re
import pandas as pd
from geopy.extra.rate_limiter import RateLimiter
from geopy.geocoders import Nominatim
from unicodedata import normalize
import folium

4 - Path to my files that will be accessed.

In [ ]:
df = pd.read_csv('/content/drive/My Drive/Geocode_Project/amostragem.csv')
df.columns

5 - A new data frame that will recive the necessary information from de first data frame.

In [ ]:
colunas = ['Address','Latitude','Longitude']
dfNew = pd.DataFrame(columns=colunas)

In [ ]:
dfNew.columns

6 - Function to clean the garbage from the lines of text.

In [ ]:
def garbageCleanerT(text):
    text = text.lower()
    text = re.sub('[0-9]', '', text)
    text = re.sub('[.]', '', text)
    text = re.sub(r"http\S+", '', text)
    text = re.sub(u'[!@#$%¨<&*(){}|/\_,.;:>?"+-]','',text)
    text = re.sub(r"(\s)+",r' ',text)
    return(normalize('NFKD', text).encode('ASCII', 'ignore').decode('ASCII'))

7 - Function to clean the garbage from the lines of numbers

In [ ]:
def garbageCleanerN(number):
    number = re.sub('[^0-9]', '', number)
    number = re.sub('[.]', '', number)
    number = re.sub(r"http\S+", '', number)
    number = re.sub(u'[!@#$%¨<&*(){}|/\_,.;:>?"+-]','',number)
    number = re.sub(r"(\s)+",r' ',number)
    return(number)

8 - Cleaning all.

In [ ]:
i = 0
for ix, ln in df.iterrows():
  try:
    dirtyText = df.Logradouro[i]
    text = garbageCleanerT(dirtyText)
    df.Logradouro[i] = text
    i += 1
  except:
    i += 1

In [ ]:
i = 0
for ix, ln in df.iterrows():
  try:
    dirtyNumber = df.numero[i]
    number = garbageCleanerN(dirtyNumber)
    df.numero[i] = number
    i += 1
  except:
    i += 1

In [ ]:
i = 0
for ix, ln in df.iterrows():
  try:
    dirtyText = df.Bairro[i]
    text = garbageCleanerT(dirtyText)
    df.Bairro[i] = text
    i += 1
  except:
    i += 1

In [ ]:
i = 0
for ix, ln in df.iterrows():
  try:
    dirtyText = df.Municipio[i]
    text = garbageCleanerT(dirtyText)
    df.Municipio[i] = text
    i += 1
  except:
    i += 1

9 - Merging columns into a single column in another data frame created earlier.

In [ ]:
dfNew['Address'] = df['Logradouro']+', '+df['Municipio']+', '+df['UF']
dfNew

10 - Calling agents who will do the search to find lat and long from a physical address.

In [ ]:
locator = Nominatim(user_agent = 'my_geocode')
geocode = RateLimiter(locator.geocode, min_delay_seconds=0.1)

11 - Doing the research.

In [ ]:
i = 0 
for ix, ln in dfNew.iterrows():
    try:
        location = locator.geocode (dfNew.Logradouro[i])    
        dfNew.Latitude[i] = (location.latitude)
        dfNew.Longitude[i] = (location.longitude)
        i += 1
    except:
        i += 1

12 - Saving one file copy with lat and long as CSV. 

In [ ]:
dfNew.to_csv('/content/drive/My Drive/Geocode_Project/out_amostragem.csv', encoding='utf-8', index=False)

13 - Clearing blank results.

In [ ]:
dfNew.dropna(subset=['Latitude'], inplace=True)

14 - Saving another clean copy of the file.

In [ ]:
dfNew.to_csv('/content/drive/My Drive/Geocode_Project/clean_out_amostragem.csv', encoding='utf-8', index=False)